## Linear Classifier in TensorFlow 
Using Low Level API in Eager Execution mode

### Load tensorflow

In [1]:
import tensorflow as tf

In [2]:
#Enable Eager Execution if using tensflow version < 2.0
#From tensorflow v2.0 onwards, Eager Execution will be enabled by default
tf.enable_eager_execution()

### Collect Data

In [0]:
from google.colab import drive
drive.mount('/gdrive')

In [3]:
import pandas as pd

In [51]:
data = pd.read_csv('prices.csv')

### Check all columns in the dataset

In [52]:
data.head()

,date,symbol,open,close,low,high,volume
0,2016-01-05 00:00:00,WLTW,123.430000,125.839996,122.309998,126.250000,2163600.0
1,2016-01-06 00:00:00,WLTW,125.239998,119.980003,119.940002,125.540001,2386400.0
2,2016-01-07 00:00:00,WLTW,116.379997,114.949997,114.930000,119.739998,2489500.0
3,2016-01-08 00:00:00,WLTW,115.480003,116.620003,113.500000,117.440002,2006300.0
4,2016-01-11 00:00:00,WLTW,117.010002,114.970001,114.089996,117.330002,1408600.0


In [53]:
data.columns

Index(['date', 'symbol', 'open', 'close', 'low', 'high', 'volume'], dtype='object')

### Drop columns `date` and  `symbol`

In [54]:
data.drop(['date','symbol'],axis=1,inplace=True)

In [55]:
data.head()

,open,close,low,high,volume
0,123.430000,125.839996,122.309998,126.250000,2163600.0
1,125.239998,119.980003,119.940002,125.540001,2386400.0
2,116.379997,114.949997,114.930000,119.739998,2489500.0
3,115.480003,116.620003,113.500000,117.440002,2006300.0
4,117.010002,114.970001,114.089996,117.330002,1408600.0


### Consider only first 1000 rows in the dataset for building feature set and target set
Target 'Volume' has very high values. Divide 'Volume' by 1000,000

In [56]:
data_df = data.head(1000)
data_df.shape

(1000, 5)

In [57]:
data_df['volume']= data_df['volume']/1000000

C:\Users\hp\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [58]:
data_df.head()

,open,close,low,high,volume
0,123.430000,125.839996,122.309998,126.250000,2.1636
1,125.239998,119.980003,119.940002,125.540001,2.3864
2,116.379997,114.949997,114.930000,119.739998,2.4895
3,115.480003,116.620003,113.500000,117.440002,2.0063
4,117.010002,114.970001,114.089996,117.330002,1.4086


### Divide the data into train and test sets

In [15]:
from sklearn.model_selection import train_test_split

In [60]:
features = data_df.drop('volume',axis=1)
target = data_df.pop('volume')

In [61]:
X_train, X_test,Y_train,Y_test = train_test_split(features,target,test_size=0.2,random_state=1)

#### Convert Training and Test Data to numpy float32 arrays


In [62]:
import numpy as np
X_train=X_train.astype(np.float32)
Y_train = Y_train.astype(np.float32)
X_test = X_test.astype(np.float32)
Y_test = Y_test.astype(np.float32)

### Normalize the data
You can use Normalizer from sklearn.preprocessing

In [63]:
from sklearn.preprocessing import Normalizer
norm = Normalizer()
X_train_scaled = norm.fit_transform(X_train)
X_test_scaled = norm.transform(X_test)

## Building the Model in tensorflow

1.Define Weights and Bias, use tf.zeros to initialize weights and Bias

In [134]:
W = tf.Variable(tf.zeros(shape=[4,1]), name="Weights")
b = tf.Variable(tf.zeros(shape=[1]),name="Bias")

2.Define a function to calculate prediction

In [75]:
def pred(x,w,b):
    y = tf.add(tf.matmul(x,w),b,name='output')
    return y

In [82]:
y_pred=pred(X_train_scaled,W,b)

3.Loss (Cost) Function [Mean square error]

In [84]:
def cost_fn(y_actual,y_pred):
    loss = tf.reduce_mean(tf.square(y_pred-y_actual),name='Loss')
    return loss

In [85]:
cost_fn(Y_train,y_pred)

<tf.Tensor: id=45, shape=(), dtype=float32, numpy=203.2221>

4.Function to train the Model

1.   Record all the mathematical steps to calculate Loss
2.   Calculate Gradients of Loss w.r.t weights and bias
3.   Update Weights and Bias based on gradients and learning rate to minimize loss

In [127]:
def TrainModel(x,y,w,b):
    with tf.GradientTape() as tape:
        tape.watch([w,b])
        y_pred=pred(x,w,b)
        loss=cost_fn(y,y_pred)
    dw, db = tape.gradient(loss, [w,b])
    w=w-0.03*dw
    b=b-0.03*db
    return[w,b,loss.numpy()]

In [117]:
W,b = TrainModel(X_train_scaled,Y_train,W,b)
print(W)
print(b)

tf.Tensor(203.2221, shape=(), dtype=float32)
tf.Tensor(
[[0.16056791]
 [0.16110753]
 [0.1588958 ]
 [0.16251586]], shape=(4, 1), dtype=float32)
tf.Tensor([0.3215619], shape=(1,), dtype=float32)


## Train the model for 100 epochs 
1. Observe the training loss at every iteration
2. Observe Test loss at every 5th iteration

In [136]:
W = tf.Variable(tf.zeros(shape=[4,1]), name="Weights")
b = tf.Variable(tf.zeros(shape=[1]),name="Bias")

In [137]:
num_epochs = 100
for epoch in range(num_epochs):
    W,b,loss = TrainModel(X_train_scaled,Y_train,W,b)
    print("Training Loss for Epoch",epoch,":",loss)
    y_test_pred = pred(X_test_scaled,W,b)
    test_loss = cost_fn(Y_test,y_test_pred)
    if(epoch%5 == 0):
        print("Test Loss for Epoch",epoch,":",test_loss.numpy())

Training Loss for Epoch 0 : 203.2221
Test Loss for Epoch 0 : 367.35556
Training Loss for Epoch 1 : 196.7424
Training Loss for Epoch 2 : 191.72452
Training Loss for Epoch 3 : 187.83855
Training Loss for Epoch 4 : 184.82933
Training Loss for Epoch 5 : 182.49902
Test Loss for Epoch 5 : 352.3129
Training Loss for Epoch 6 : 180.69427
Training Loss for Epoch 7 : 179.29675
Training Loss for Epoch 8 : 178.21437
Training Loss for Epoch 9 : 177.37633
Training Loss for Epoch 10 : 176.72731
Test Loss for Epoch 10 : 348.37354
Training Loss for Epoch 11 : 176.22466
Training Loss for Epoch 12 : 175.83534
Training Loss for Epoch 13 : 175.53392
Training Loss for Epoch 14 : 175.30061
Training Loss for Epoch 15 : 175.11978
Test Loss for Epoch 15 : 347.40915
Training Loss for Epoch 16 : 174.97984
Training Loss for Epoch 17 : 174.8715
Training Loss for Epoch 18 : 174.78748
Training Loss for Epoch 19 : 174.72258
Training Loss for Epoch 20 : 174.6721
Test Loss for Epoch 20 : 347.21
Training Loss for Epoch 21

### Get the shapes and values of W and b

In [140]:
print("Values of W are",W.numpy())

Values of W are [[1.3380946]
 [1.3425944]
 [1.3241585]
 [1.3543311]]


In [142]:
print("Shape of W is",W.get_shape())

Shape of W is (4, 1)


In [143]:
print("Values of b are",b.numpy())

Values of b are [2.6797438]


In [144]:
print("Shape of b is",b.get_shape())

Shape of b is (1,)


### Model Prediction on 1st Examples in Test Dataset

In [148]:
y_test_pred = pred(X_test_scaled,W,b)
print(y_test_pred)

tf.Tensor(
[[5.3594174]
 [5.3593993]
 [5.3593807]
 [5.3593283]
 [5.3594074]
 [5.3587127]
 [5.3593946]
 [5.3593664]
 [5.359397 ]
 [5.35941  ]
 [5.359212 ]
 [5.3593764]
 [5.35937  ]
 [5.359401 ]
 [5.3592696]
 [5.359364 ]
 [5.3593955]
 [5.3591695]
 [5.359418 ]
 [5.3593216]
 [5.3593874]
 [5.3594127]
 [5.359312 ]
 [5.35938  ]
 [5.359402 ]
 [5.359106 ]
 [5.359399 ]
 [5.3593984]
 [5.3593893]
 [5.3593693]
 [5.359399 ]
 [5.3593864]
 [5.359329 ]
 [5.3594007]
 [5.3593273]
 [5.3592567]
 [5.3594046]
 [5.359394 ]
 [5.3592877]
 [5.359416 ]
 [5.3594027]
 [5.3593884]
 [5.3593645]
 [5.359391 ]
 [5.3594112]
 [5.359383 ]
 [5.3593884]
 [5.3594112]
 [5.3593802]
 [5.359376 ]
 [5.3594103]
 [5.359417 ]
 [5.359393 ]
 [5.359406 ]
 [5.359313 ]
 [5.359352 ]
 [5.359391 ]
 [5.3593845]
 [5.3594017]
 [5.3593693]
 [5.3594055]
 [5.359356 ]
 [5.359413 ]
 [5.359376 ]
 [5.359413 ]
 [5.359383 ]
 [5.3593864]
 [5.359373 ]
 [5.359401 ]
 [5.3593993]
 [5.3594074]
 [5.359272 ]
 [5.3594027]
 [5.359398 ]
 [5.359416 ]
 [5.359374 ]
 

## Classification using tf.Keras

In this exercise, we will build a Deep Neural Network using tf.Keras. We will use Iris Dataset for this exercise.

### Load tensorflow if not done already

### Load the given Iris data using pandas (Iris.csv)

In [9]:
iris_df = pd.read_csv('iris.csv')
iris_df.head()

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,1,5.1,3.5,1.4,0.2,Iris-setosa
1,2,4.9,3.0,1.4,0.2,Iris-setosa
2,3,4.7,3.2,1.3,0.2,Iris-setosa
3,4,4.6,3.1,1.5,0.2,Iris-setosa
4,5,5.0,3.6,1.4,0.2,Iris-setosa


### Target set has different categories. So, Label encode them. And convert into one-hot vectors using get_dummies in pandas.

In [10]:
iris_df=pd.get_dummies(iris_df)
iris_df.head()

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species_Iris-setosa,Species_Iris-versicolor,Species_Iris-virginica
0,1,5.1,3.5,1.4,0.2,1,0,0
1,2,4.9,3.0,1.4,0.2,1,0,0
2,3,4.7,3.2,1.3,0.2,1,0,0
3,4,4.6,3.1,1.5,0.2,1,0,0
4,5,5.0,3.6,1.4,0.2,1,0,0


In [11]:
iris_df.drop('Id',axis=1,inplace=True)

### Splitting the data into feature set and target set

In [14]:
features = iris_df.iloc[:,:-3]
target =  iris_df.iloc[:,-3:]

### Divide the dataset into Training and test (70:30)

In [35]:
X_train, X_test,Y_train,Y_test = train_test_split(features,target,test_size=0.3,random_state=1)

###  Building Model in tf.keras

Build a Linear Classifier model  <br>
1.  Use Dense Layer  with input shape of 4 (according to the feature set) and number of outputs set to 3<br> 
2. Apply Softmax on Dense Layer outputs <br>
3. Use SGD as Optimizer
4. Use categorical_crossentropy as loss function 

In [55]:
#Initialize Sequential Graph (model)
model = tf.keras.models.Sequential()

#Normalize input data
model.add(tf.keras.layers.BatchNormalization(input_shape=(4,)))

#Add Dense layer for prediction - Keras declares weights and bias automatically
model.add(tf.keras.layers.Dense(3,activation='softmax'))

#Compile the model - add Loss and Gradient Descent optimizer
model.compile(optimizer='sgd', loss='categorical_crossentropy')

### Model Training 

In [56]:
model.fit(X_train, Y_train, validation_data=[X_test,Y_test], epochs=50)

Train on 105 samples, validate on 45 samples
Epoch 1/50
105/105 [==============================] - 1s 6ms/sample - loss: 1.9260 - val_loss: 5.5160
Epoch 2/50
105/105 [==============================] - 0s 105us/sample - loss: 1.8343 - val_loss: 4.9843
Epoch 3/50
105/105 [==============================] - 0s 133us/sample - loss: 1.7550 - val_loss: 4.5051
Epoch 4/50
105/105 [==============================] - 0s 104us/sample - loss: 1.6365 - val_loss: 4.1040
Epoch 5/50
105/105 [==============================] - 0s 152us/sample - loss: 1.5942 - val_loss: 3.7374
Epoch 6/50
105/105 [==============================] - 0s 133us/sample - loss: 1.5088 - val_loss: 3.4200
Epoch 7/50
105/105 [==============================] - 0s 123us/sample - loss: 1.4550 - val_loss: 3.1427
Epoch 8/50
105/105 [==============================] - 0s 142us/sample - loss: 1.3935 - val_loss: 2.8927
Epoch 9/50
105/105 [==============================] - 0s 104us/sample - loss: 1.3461 - val_loss: 2.6695
Epoch 10/50
105/105 [

### Model Prediction

In [45]:
y_pred = model.predict(X_test)
print(y_pred)

[[0.7681112  0.21424845 0.01764031]
 [0.2963103  0.39376915 0.30992055]
 [0.13776132 0.3746807  0.48755795]
 [0.74280417 0.2271695  0.03002633]
 [0.03111684 0.17345972 0.79542345]
 [0.11753712 0.2760616  0.60640126]
 [0.01554077 0.13887924 0.84558004]
 [0.7678046  0.19633025 0.03586507]
 [0.7825929  0.1914627  0.02594443]
 [0.01473409 0.11387464 0.87139124]
 [0.1938965  0.33153817 0.47456536]
 [0.7490835  0.21748151 0.03343492]
 [0.01982216 0.14695773 0.83322006]
 [0.13459246 0.32192758 0.5434799 ]
 [0.1279419  0.29916334 0.57289475]
 [0.8040438  0.17286852 0.02308768]
 [0.20824164 0.38351998 0.40823838]
 [0.1521781  0.26586562 0.5819563 ]
 [0.6775743  0.2825338  0.03989189]
 [0.74788874 0.22804391 0.02406739]
 [0.22235619 0.3285763  0.4490675 ]
 [0.16095018 0.25136322 0.58768654]
 [0.06760549 0.26653448 0.66586   ]
 [0.759589   0.21304797 0.02736294]
 [0.04529205 0.20683925 0.74786866]
 [0.19991623 0.3424692  0.45761457]
 [0.8192916  0.16452    0.01618841]
 [0.7885734  0.18631223 0.02

In [72]:
y_pred_class = model.predict_classes(X_test)
print(y_pred_class)

[0 1 2 0 2 2 2 0 0 2 2 0 2 2 2 0 2 2 0 0 2 2 2 0 2 2 0 0 2 2 2 2 2 2 2 0 2
 0 2 2 2 0 2 2 2]


### Save the Model

In [46]:
model.save('iris_output.h5')

### Build and Train a Deep Neural network with 2 hidden layer  - Optional - For Practice

Does it perform better than Linear Classifier? What could be the reason for difference in performance?

In [69]:
#Initialize Sequential Graph (model)
model = tf.keras.models.Sequential()

#Normalize input data
model.add(tf.keras.layers.BatchNormalization(input_shape=(4,)))

#Add Dense layer for prediction - Keras declares weights and bias automatically
model.add(tf.keras.layers.Dense(6,activation='relu'))

#model.add(tf.keras.layers.Dense(3,activation='relu'))

model.add(tf.keras.layers.Dense(3,activation='softmax'))

#Compile the model - add Loss and Gradient Descent optimizer
model.compile(optimizer='sgd', loss='categorical_crossentropy')

In [70]:
model.fit(X_train, Y_train, validation_data=[X_test,Y_test], epochs=50)

Train on 105 samples, validate on 45 samples
Epoch 1/50
105/105 [==============================] - 1s 9ms/sample - loss: 1.1092 - val_loss: 1.1054
Epoch 2/50
105/105 [==============================] - ETA: 0s - loss: 1.096 - 0s 142us/sample - loss: 1.0715 - val_loss: 1.0697
Epoch 3/50
105/105 [==============================] - 0s 142us/sample - loss: 1.0289 - val_loss: 1.0418
Epoch 4/50
105/105 [==============================] - 0s 152us/sample - loss: 0.9943 - val_loss: 1.0137
Epoch 5/50
105/105 [==============================] - 0s 161us/sample - loss: 0.9602 - val_loss: 0.9854
Epoch 6/50
105/105 [==============================] - 0s 171us/sample - loss: 0.9232 - val_loss: 0.9750
Epoch 7/50
105/105 [==============================] - 0s 171us/sample - loss: 0.8948 - val_loss: 0.9633
Epoch 8/50
105/105 [==============================] - 0s 152us/sample - loss: 0.8691 - val_loss: 0.9510
Epoch 9/50
105/105 [==============================] - 0s 142us/sample - loss: 0.8413 - val_loss: 0.93